In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import load_iris, load_digits, load_wine
from sklearn.cluster import KMeans
from scipy.optimize import minimize, differential_evolution
from scipy.spatial.distance import pdist, squareform
import pandas as pd
from fitting.manifold_learning import ManifoldFitter
from fitting.energy_learning import practical_examples, advanced_energy_analysis, ManifoldEnergyIntegration


In [ ]:

fitter = ManifoldFitter()

print("=== Fitting Real Data onto Manifolds ===\n")

# Load sample datasets
datasets = fitter.load_sample_datasets()

for dataset_name, dataset_info in datasets.items():
    print(f"\n--- Analyzing {dataset_name}: {dataset_info['description']} ---")

    data = dataset_info['data']

    # Preprocessing
    if data.shape[1] > 3:
        # Reduce dimension to 3D
        pca = PCA(n_components=3)
        data_3d = pca.fit_transform(data)
        print(f"Dimension reduction: {data.shape[1]} -> 3")
        print(f"Variance preserved: {sum(pca.explained_variance_ratio_):.3f}")
    else:
        data_3d = data

    # Standardize
    data_scaled, scaler = fitter.preprocess_data(data_3d, method='standard')

    # Estimate the best manifold
    best_manifold, scores = fitter.estimate_manifold_type(data_scaled)
    print(f"\nManifold scores:")
    for manifold, score in scores.items():
        print(f"  {manifold}: {score:.3f}")
    print(f"Suggestion: {best_manifold}")

    # Fit to all manifolds
    results, sorted_results = fitter.fit_data_to_manifolds(data_scaled)

    print(f"\nFit results (sorted by error):")
    for manifold, result in sorted_results:
        if 'error' in result and result['error'] != float('inf'):
            print(f"  {manifold}: Error = {result['error']:.6f}")
            if 'aic' in result:
                print(f"    AIC = {result['aic']:.2f}, BIC = {result['bic']:.2f}")

    # Display parameters of the best fit
    if sorted_results:
        best_manifold_result = sorted_results[0]
        manifold_name = best_manifold_result[0]
        manifold_data = best_manifold_result[1]

        if 'params' in manifold_data:
            print(f"\nParameters for {manifold_name}:")
            for param_name, param_value in manifold_data['params'].items():
                if isinstance(param_value, np.ndarray):
                    print(f"  {param_name}: {param_value}")
                else:
                    print(f"  {param_name}: {param_value:.4f}")

    # Plot results (optional)
    # fitter.visualize_fits(data_scaled, results, dataset_name)

    print("-" * 60)

print("\n=== Example with Custom Data ===")

# Generate synthetic data for testing
np.random.seed(42)

# Spherical data
theta = np.random.uniform(0, np.pi, 300)
phi = np.random.uniform(0, 2*np.pi, 300)
sphere_data = np.column_stack([
    2 * np.sin(theta) * np.cos(phi) + np.random.normal(0, 0.1, 300),
    2 * np.sin(theta) * np.sin(phi) + np.random.normal(0, 0.1, 300),
    2 * np.cos(theta) + np.random.normal(0, 0.1, 300)
])

print("Testing with spherical data:")
results, sorted_results = fitter.fit_data_to_manifolds(sphere_data)

for manifold, result in sorted_results:
    if 'error' in result and result['error'] != float('inf'):
        print(f"  {manifold}: Error = {result['error']:.6f}")

# Check if the fit is correct
best_result = sorted_results[0]
if best_result[0] == 'sphere':
    sphere_params = best_result[1]['params']
    estimated_center = sphere_params['center']
    estimated_radius = sphere_params['radius']

    print(f"\nEstimated center: {estimated_center}")
    print(f"Estimated radius: {estimated_radius:.3f}")
    print("(Expected: center ≈ [0,0,0], radius ≈ 2)")

In [ ]:

print("🚀 Starting Manifold-Energy Learning Combination Analysis")
print("=" * 60)

# Practical examples
detailed_results, best_manifold, train_data, ood_data = practical_examples()

# Advanced analysis
energy_model, stability = advanced_energy_analysis()

print(f"\n" + "=" * 60)
print("📊 Final Summary")
print("=" * 60)

if detailed_results and best_manifold:
    print(f"✅ Best manifold: {best_manifold}")
    print(f"✅ AUC of the best model: {detailed_results[best_manifold]['ood_auc']:.4f}")

if stability:
    print(f"✅ Model Stability: {np.mean(stability):.4f} ± {np.std(stability):.4f}")

print(f"\n💡 Conclusion:")
print(f"  🔹 Combining manifold and energy is better than separate methods")
print(f"  🔹 Choosing the right manifold is very important")
print(f"  🔹 The number of energy centers has a significant impact")
print(f"  🔹 A stable and reliable method")

print(f"\n🎯 Practical Applications:")
print(f"  📈 Anomaly Detection in complex systems")
print(f"  📈 Quality Control in manufacturing")
print(f"  📈 Medical Diagnosis")
print(f"  📈 Fraud Detection")
print(f"  📈 Computer Vision OOD Detection")

print(f"\n🔬 To draw plots, uncomment the visualization code!")

# To draw:
integrator = ManifoldEnergyIntegration()
integrator.visualize_manifold_energy(train_data, ood_data, 'sphere')

In [ ]:
"""
Main execution function demonstrating Energy-Based Bayesian Deep Learning
"""
from fitting.energy_based_bnn import *

print("="*80)
print("🔥 ENERGY-BASED BAYESIAN DEEP LEARNING")
print("Integration of Energy Functions with Bayesian Neural Networks")
print("="*80)

# 1. Initialize model and trainer
print("\n🚀 Step 1: Model Initialization")
model = EnergyBayesianNetwork(
    input_dim=2,
    hidden_dims=[64, 32, 16],
    output_dim=2,
    prior_std=1.0,
    energy_reg=0.1
)

optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.8, patience=15
)

trainer = EnergyBayesianTrainer(model, optimizer, scheduler)

# 2. Prepare data
print("\n📊 Step 2: Data Preparation")
X_train, X_test, y_train, y_test = trainer.prepare_data(
    data_type='moons', n_samples=1000, noise=0.1, add_outliers=True
)

# 3. Train model
print("\n🔥 Step 3: Model Training")
training_history = trainer.train(X_train, y_train, n_epochs=300, verbose=True)

# 4. Initialize analyzer
print("\n🔍 Step 4: Analysis Initialization")
analyzer = EnergyBayesianAnalyzer(model, X_train, y_train, X_test, y_test)
analyzer.training_history = training_history  # Store training history for visualization

# 5. Comprehensive analysis
print("\n📈 Step 5: Comprehensive Analysis")

# Energy landscape visualization
energy_grid, uncertainty_grid, prob_grid = analyzer.visualize_energy_landscape()

# OOD detection analysis
ood_results = analyzer.analyze_ood_detection(n_samples=50)

# Energy centers analysis
centers_results = analyzer.analyze_energy_centers()

# Comparison with standard Bayesian NN
comparison_results = analyzer.compare_with_standard_bayesian(n_epochs=100)

# 6. Final summary
print("\n" + "="*80)
print("🎊 COMPREHENSIVE RESULTS SUMMARY")
print("="*80)

print(f"\n📊 Model Performance:")
print(f"  ✅ Final Training Accuracy: {training_history['accuracies'][-1]:.4f}")
print(f"  ✅ Test Accuracy: {comparison_results['energy_accuracy']:.4f}")
print(f"  ✅ Average Uncertainty: {comparison_results['energy_uncertainty']:.4f}")

print(f"\n🚨 Out-of-Distribution Detection:")
print(f"  ✅ Energy-based AUC: {ood_results['auc_energy']:.3f}")
print(f"  ✅ Uncertainty-based AUC: {ood_results['auc_uncertainty']:.3f}")
print(f"  ✅ Energy separation ratio: {np.mean(ood_results['ood_energies']) / np.mean(ood_results['id_energies']):.2f}x")

print(f"\n🎯 Energy Centers Analysis:")
print(f"  ✅ Global energy centers: {len(centers_results['global_centers'])}")
print(f"  ✅ Average distance to data: {np.mean(centers_results['distances_to_data']):.4f}")
print(f"  ✅ Energy landscape efficiency: {(1 - np.mean(centers_results['distances_to_data']) / np.max(centers_results['distances_to_data'])) * 100:.1f}%")

print(f"\n⚔️ Comparison with Standard Bayesian NN:")
print(f"  ✅ Accuracy improvement: {comparison_results['improvement_percentage']:+.1f}%")
print(f"  ✅ Energy-based model accuracy: {comparison_results['energy_accuracy']:.4f}")
print(f"  ✅ Standard model accuracy: {comparison_results['standard_accuracy']:.4f}")
print(f"  ✅ OOD uncertainty improvement: {(comparison_results['energy_ood_uncertainty'] / comparison_results['standard_ood_uncertainty']):.2f}x")

print(f"\n🧠 Key Innovations Demonstrated:")
print(f"  ✅ Energy-based regularization with adaptive centers")
print(f"  ✅ Multi-level uncertainty quantification")
print(f"  ✅ Physics-inspired learning dynamics")
print(f"  ✅ Robust out-of-distribution detection")
print(f"  ✅ Interpretable energy landscapes")

print(f"\n🚀 Practical Applications:")
print(f"  • Autonomous systems with safety-critical decisions")
print(f"  • Medical diagnosis with uncertainty-aware predictions")
print(f"  • Financial risk assessment with anomaly detection")
print(f"  • Scientific modeling with principled uncertainty")
print(f"  • Quality control in manufacturing processes")

print(f"\n💡 Mathematical Framework:")
print(f"  • Variational Bayesian inference: ELBO optimization")
print(f"  • Energy function: E(x,θ) = -log p(x|θ)")
print(f"  • Combined objective: L = CE + βKL + λE")
print(f"  • Adaptive energy centers with learnable scales")
print(f"  • Monte Carlo uncertainty estimation")

print(f"\n🎯 Conclusion:")
print(f"✨ Energy-Based Bayesian Deep Learning successfully combines:")
print(f"   • Principled uncertainty quantification")
print(f"   • Physics-inspired regularization")
print(f"   • Robust anomaly detection")
print(f"   • Interpretable decision boundaries")
print(f"   • Superior performance on challenging tasks")


In [ ]:
from mgem.demo import *
# Run the demo
print("Starting Multi-Geometry Energy Model Demo...")
results = quick_demo()
print("Demo completed! Check the results dictionary for all outputs.")